In [2]:
import pandas as pd

# third-party library imports
from bs4 import BeautifulSoup
from glob import glob

In [2]:
def soup_html_parser(file_path):
    """Convert HTML text into a data object


    Args:
        file_path (str): file path

    Returns:
        bs4.BeautifulSoup: HTML syntax parsed
    """
    with open(file_path, encoding='utf-8') as file:
        text = file.read()

    return BeautifulSoup(text, 'html.parser')

In [1]:
def rename_columns(dataset):
    """rename columns name(dataframe header) from arabic into english

    Args:
        dataset (DataFrame): Cars dataset
    """
    dataset.columns = ['car_color', 'fuel_type', 'car_type', 'car_license',
                        'gear_type', 'windows', 'motor_power', 'speedometer', 'passengers_number', 
                        'payment_method', 'car_state', 'ex_owners', 'additions', 'price']

In [4]:
with open('../data_files/valid_files.txt', 'r') as f:
    valid_files = f.read().splitlines()

In [6]:
with open('valid_files.txt', 'r') as f:
    valid_files = f.read().splitlines()

In [7]:
for indx, file_path in enumerate(valid_files):
    valid_files[indx] = file_path.replace('\\','/')

In [9]:
features = ['لون السيارة', 'نوع الوقود', 'أصل السيارة', 'رخصة السيارة',
            'نوع الجير', 'الزجاج', 'قوة الماتور', 'عداد السيارة',
            'عدد الركاب', 'وسيلة الدفع', 'معروضة', 'أصحاب سابقون', 'إضافات', 'price']
dataset = pd.DataFrame(columns=features)
int_features = ['قوة الماتور', 'عداد السيارة']

for file_path in valid_files:
    html_parsed = soup_html_parser(file_path)
    list_sample_features = []
    dict_row_df = dict.fromkeys(features)

    # extract <table> elements needed from the file
    tables_extracted = html_parsed.findAll('table', {'class':['list_ads', 'driving-table']})

    # extraxt <td> elements included features from the second extracted table
    sample_features_html = tables_extracted[1].findAll('td')

    # extraxt target variable from the first extracted table
    price_sample = int((tables_extracted)[0].
                    find('h5', attrs={"class":"post-price"}).
                    text.split('\n')[0].split(' ')[0])

    # extraxt features names from each html element and append it to a list 
    for feature_value in sample_features_html:
        list_sample_features.append(feature_value.text)

    # create dictionary of features values for one sample
    for indx, val in enumerate(list_sample_features):
        if val in features:
            dict_row_df[val] = list_sample_features[indx+1]
    
    # set dependent variable value in dictionary for one sample
    dict_row_df['price'] = price_sample

    # insert a new sample to the dataframe
    dataset.loc[len(dataset)] = dict_row_df


In [5]:
rename_columns(dataset)

In [8]:
dataset.head()

,car_color,fuel_type,car_type,car_license,gear_type,windows,motor_power,speedometer,passengers_number,payment_method,car_state,ex_owners,additions,price
0,أبيض عاجي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,2000,75000,4+1,نقدا فقط,للبيع فقط,يد اولى,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,100000
1,سكني,ديزل,خصوصي,فلسطينية,نصف اوتوماتيك,الكتروني,2500,130000,7+1,إمكانية التقسيط,للبيع أو التبديل,2,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,60000
2,سكني,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1600,NaN,NaN,نقدا فقط,للبيع فقط,NaN,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nمسجل CD\n...,43500
3,بيج,بنزين,خصوصي,فلسطينية,عادي,يدوي,906,شغال,4+1,إمكانية التقسيط,للبيع فقط,00,\n\nمُكيّف\nمسجل CD\n\n,5500
4,فضي,بنزين,خصوصي,فلسطينية,اوتوماتيك,الكتروني,1200,38000,4+1,نقدا فقط,للبيع فقط,NaN,\n\nمُكيّف\nإغلاق مركزي\nجهاز إنذار\nجنطات مغن...,54000


In [7]:
dataset.to_csv('./data_df/dataset_df__13-features__price.csv', index=False)